# PROJET FINAL

## Importations de librairies

In [315]:
from shutil import copyfileobj
from urllib import request
import numpy as np
import pandas as pd
from datetime import datetime
import uuid
import xlwings as xw
import requests
from bs4 import BeautifulSoup
import os

## 1.Acquisition et chargement des données  

In [162]:
#Initialisation des variables
debut = datetime(2020,11,8)
arrivee = datetime(2021,1,26)
heure = ['04','08','11','14','17','21']
heures = ['14','17','21']
link = r'C:\Users\Abo\OneDrive\Documents\MS\KitBigData\Kit-Big-Data - Copie'

In [163]:
#Fonction date int

def datestr(date):
    date_str = date.strftime('%Y%m%d')
    return date_str     


def arriveestr(date):
    arrivees = arrivee.strftime('%Y%m%d')
    return arrivees   

In [169]:
#Intialisation des dates
dates = []
date = debut

while date <= arrivee:  
    str_d = datestr(date)
    dates.append(str_d)
    date = date + timedelta(days=1)

In [149]:
#Téléchargement des fichiers
if not os.listdir(link):
    for d in dates:
        for h in heure:
            url = f'https://www.vendeeglobe.org/download-race-data/vendeeglobe_{d}_{h}0000.xlsx'
            wget.download(url,out=link)
    for h in heures:
        url = f'https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_{h}0000.xlsx'
        wget.download(url,out=link)

In [201]:
#Prépartaion des données
def cleaning(date,heure):
    
    url = f"data/vendeeglobe_{date}_{heure}0000.xlsx"
    
    with xw.Book(url, mode="r") as fichier:
        sheet1 = fichier.sheets[0]
        df = sheet1.cells.options('df').value
        df.iloc[3,:3] = df.iloc[2,:3]
        df.iloc[3,-2:] = df.iloc[2,-2:]
        df = df.iloc[3:-4]
        
        df.iloc[0,0]=df.iloc[0,0].replace('\r\nRank','')
        df.iloc[0] = df.iloc[0].str.split('\r\n').str[0]
        df.iloc[0,6:10] = df.iloc[0,6:10]+"_since30min"
        df.iloc[0,10:14] = df.iloc[0,10:14]+"_lastReport"
        df.iloc[0,14:18] = df.iloc[0,14:18]+"_since24h"
        df.columns = df.iloc[0]
        df = df.iloc[1:]
        
        df['Nationalité'] = df.iloc[:,1].str.split(' ').str[0]
        df['Nationalité'] = df.iloc[:,-1].str.split('\r\n').str[-1]
        df['Voile'] = df.iloc[:,1].str.split(' ').str[1]
        df['Skipper'] = df.iloc[:,2].str.split('\r\n').str[0]
        df['Bateau'] = df.iloc[:,2].str.split('\r\n').str[1]
        df = df.drop(['Nat. / Voile', 'Skipper / Bateau'], axis=1)
        jour = datetime.strptime(date, '%Y%m%d')
        df['Date'] = jour
        df = df.reset_index(drop=True)
        
        return df 

In [ ]:
# Affichage des donées en un seul dateframe 

df_cleaning = pd.DataFrame()

for h in heures:
        data = cleaning(date,h)
        data = data.dropna()
        df_cleaning = pd.concat([df_cleaning,data],axis=0)
        
for d in dates:
    for h in heure:
        data = cleaning(d,h)
        data = data.dropna()
        df_cleaning = pd.concat([df_cleaning,data],axis=0)
        
df_cleaning.reset_index(drop=True)
df_cleaning.head()


# 2. Préparation des données

## Préparation des données relatives aux classements.

In [102]:
#Beautifulsoup
url = "https://www.vendeeglobe.org/fr/glossaire"
soup = BeautifulSoup(requests.get(url).content)

In [211]:
#Scrapping bateau et skipper
bateaux = soup.find_all('h3', {"class","boats-list__boat-name"})
bateau = []
skipper = []
for boat in bateaux:
    bateau_nom = boat.text
    bateau.append(bateau_nom)
    skippers = boat.parent()[1].text.title()
    skipper.append(skippers)

In [220]:
boat_list = soup.find_all('ul', {"class","boats-list__popup-specs-list"})
dico1 = []
for boat in boat_car:
    dico2 = {}
    li_tag = boat.find_all('li')
    for tag in li_tag :
        dico2[tag.text.split(':')[0]] = tag.text.split(':')[1]
    dico1.append(dico2)
df_boat = pd.DataFrame(dico1)

bateau = pd.DataFrame(bateau,columns=['Nom du bateau'])
skipper = pd.DataFrame(skipper,columns = ['Skipper'])
df_bateau = pd.concat([bateau,skipper,df_boat],axis=1)

df_bateau

,Nom du bateau,Skipper,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,NEWREST - ART & FENÊTRES,Fabrice Amedeo,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,PURE - Best Western®,Romain Attanasio,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2
2,TSE - 4MYPLANET,Alexia Barrier,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeu...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2
3,Maître CoQ IV,Yannick Bestaven,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2
4,CHARAL,Jérémie Beyou,08,NaN,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2
5,LA MIE CÂLINE - ARTISANS ARTIPÔLE,Arnaud Boissières,FRA 14,"Ecover3, Président, Gamesa, Kilcullen Voyager...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,03 Août 2007,"18,28 m","5,65 m","4,50 m","7,9 tonnes",foils,29 m,basculante avec vérin,300 m²,610 m²
6,BUREAU VALLEE 2,Louis Burton,18,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,09 Juin 2015,"18,28 m","5,80 m","4,50 m","7,6 t",foils,28 m,acier,300 m2,600 m2
7,ONE PLANET ONE OCEAN,Didac Costa,ESP 33,Kingfisher - Educacion sin Fronteras - Forum ...,Owen Clarke Design,Martens Yachts,02 Février 2000,"18,28 m","5,30 m","4,50 m","8,9 t",2,26 m,acier,240 m2,470 m2
8,GROUPE SÉTIN,Manuel Cousin,FRA 71,"Paprec-Virbac2, Estrella Damm, We are Water, ...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),02 Février 2007,"18,28 m","5,80 m","4,50 m",9 t,2 asymétriques,"28,50",basculante sur vérin hydraulique,270 m2,560 m2
9,BANQUE POPULAIRE X,Clarisse Cremer,FRA30,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,01 Mars 2011,"18,28 m","5,70 m","4,5 m","7,7 t",2,29 m,acier forgé,340 m2,570 m2


In [221]:
#Suppresion des valeurs nulles
df_bateau = df_bateau.drop(16, axis=0)
df_bateau = df_bateau.reset_index(drop=True)

In [223]:
#Tableau nettoyé
df_bateau

,Nom du bateau,Skipper,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,NEWREST - ART & FENÊTRES,Fabrice Amedeo,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,PURE - Best Western®,Romain Attanasio,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2
2,TSE - 4MYPLANET,Alexia Barrier,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeu...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2
3,Maître CoQ IV,Yannick Bestaven,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2
4,CHARAL,Jérémie Beyou,08,NaN,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2
5,LA MIE CÂLINE - ARTISANS ARTIPÔLE,Arnaud Boissières,FRA 14,"Ecover3, Président, Gamesa, Kilcullen Voyager...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,03 Août 2007,"18,28 m","5,65 m","4,50 m","7,9 tonnes",foils,29 m,basculante avec vérin,300 m²,610 m²
6,BUREAU VALLEE 2,Louis Burton,18,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,09 Juin 2015,"18,28 m","5,80 m","4,50 m","7,6 t",foils,28 m,acier,300 m2,600 m2
7,ONE PLANET ONE OCEAN,Didac Costa,ESP 33,Kingfisher - Educacion sin Fronteras - Forum ...,Owen Clarke Design,Martens Yachts,02 Février 2000,"18,28 m","5,30 m","4,50 m","8,9 t",2,26 m,acier,240 m2,470 m2
8,GROUPE SÉTIN,Manuel Cousin,FRA 71,"Paprec-Virbac2, Estrella Damm, We are Water, ...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),02 Février 2007,"18,28 m","5,80 m","4,50 m",9 t,2 asymétriques,"28,50",basculante sur vérin hydraulique,270 m2,560 m2
9,BANQUE POPULAIRE X,Clarisse Cremer,FRA30,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,01 Mars 2011,"18,28 m","5,70 m","4,5 m","7,7 t",2,29 m,acier forgé,340 m2,570 m2


## Extraction des caractéristiques techniques de chacun des voiliers. 

In [291]:
#Préparation des données

k = [7,8,9,10,11,12,14,15]

voil = df_bateau.iloc[:,n].copy()

pd.options.mode.chained_assignment = None

In [292]:
voil = voil.stack().str.replace(',','.').unstack()

voil.iloc[14,3]=None
voil.iloc[24,3]=None

In [293]:
titr = ['Longueur ', 'Largeur ', 'Tirant d\'eau ', 
       'Hauteur mât ','Surface de voiles au près ', 'Surface de voiles au portant ']
for t in titr:
    voil[t] = [float(v[0]) for v in voil[t].str.split('m')]
    
depl = voil['Déplacement (poids) '].str.split('t')
for i in range(len(voil)):
    if voil['Déplacement (poids) '][i]:
        voil['Déplacement (poids) '][i] = float(depl[i][0])
    else:
        pass
    
voil



,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Hauteur mât,Surface de voiles au près,Surface de voiles au portant,Nombre de dérives
0,18.28,5.85,4.5,7.0,29.0,320.0,570.0,foils
1,18.28,5.80,4.5,9.0,28.0,280.0,560.0,2
2,18.28,5.54,4.5,9.0,29.0,260.0,580.0,2
3,18.28,5.80,4.5,8.0,29.0,310.0,550.0,foils
4,18.28,5.85,4.5,8.0,29.0,320.0,600.0,foils
5,18.28,5.65,4.5,7.9,29.0,300.0,610.0,foils
6,18.28,5.80,4.5,7.6,28.0,300.0,600.0,foils
7,18.28,5.30,4.5,8.9,26.0,240.0,470.0,2
8,18.28,5.80,4.5,9.0,28.5,270.0,560.0,2 asymétriques
9,18.28,5.70,4.5,7.7,29.0,340.0,570.0,2


In [307]:
#Ajout de valeurs manquantes par avec des valeurs similaires par rapport à la hauteur du mât et le poids
 
index = list(range(32))
index.remove(14)
index.remove(24)

voil.iloc[14,3] = 7.9
voil.iloc[24,3] = 7.7

voil


,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Hauteur mât,Surface de voiles au près,Surface de voiles au portant,Nombre de dérives
0,18.28,5.85,4.5,7.0,29.0,320.0,570.0,foils
1,18.28,5.80,4.5,9.0,28.0,280.0,560.0,2
2,18.28,5.54,4.5,9.0,29.0,260.0,580.0,2
3,18.28,5.80,4.5,8.0,29.0,310.0,550.0,foils
4,18.28,5.85,4.5,8.0,29.0,320.0,600.0,foils
5,18.28,5.65,4.5,7.9,29.0,300.0,610.0,foils
6,18.28,5.80,4.5,7.6,28.0,300.0,600.0,foils
7,18.28,5.30,4.5,8.9,26.0,240.0,470.0,2
8,18.28,5.80,4.5,9.0,28.5,270.0,560.0,2 asymétriques
9,18.28,5.70,4.5,7.7,29.0,340.0,570.0,2


## Rapprochement des données des voiliers avec celle des skippers. 

In [309]:
voil['Skipper'] = df_bateau['Skipper']
voil

,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Hauteur mât,Surface de voiles au près,Surface de voiles au portant,Nombre de dérives,Skipper
0,18.28,5.85,4.5,7.0,29.0,320.0,570.0,foils,Fabrice Amedeo
1,18.28,5.80,4.5,9.0,28.0,280.0,560.0,2,Romain Attanasio
2,18.28,5.54,4.5,9.0,29.0,260.0,580.0,2,Alexia Barrier
3,18.28,5.80,4.5,8.0,29.0,310.0,550.0,foils,Yannick Bestaven
4,18.28,5.85,4.5,8.0,29.0,320.0,600.0,foils,Jérémie Beyou
5,18.28,5.65,4.5,7.9,29.0,300.0,610.0,foils,Arnaud Boissières
6,18.28,5.80,4.5,7.6,28.0,300.0,600.0,foils,Louis Burton
7,18.28,5.30,4.5,8.9,26.0,240.0,470.0,2,Didac Costa
8,18.28,5.80,4.5,9.0,28.5,270.0,560.0,2 asymétriques,Manuel Cousin
9,18.28,5.70,4.5,7.7,29.0,340.0,570.0,2,Clarisse Cremer
